In [820]:
from numpy import genfromtxt
import numpy as np
import os
from collections import defaultdict
from itertools import permutations, repeat
import random
from numpy import random as numpy_random

## Setup

In [821]:
# The velocity is also discrete, a number of grid cells 
# moved horizontally and vertically per time step.

# Actions:
# The actions are increments to the velocity components. 
# Each may be changed by +1, −1, or 0 in one step, 
# for a total of nine actions.

# Both velocity components are restricted to be nonnegative 
# and less than 5, and they cannot both be zero except at the 
# starting line.

# Episodes:
# Each episode begins in one of the randomly selected 
# start states with both velocity components zero and 
# ends when the car crosses the finish line.

# Rewards:
# The rewards are −1 for each step until the car crosses 
# the finish line.

# If the car hits the track boundary, it is moved back 
# to a random position on the starting line, both velocity 
# components are reduced to zero, and the episode continues.

# With probability 0.1 at each time step the velocity increments are both zero, 
# independently of the intended increments.

## Task

In [823]:
# Load map1
CELL_TYPE_WALL = 0  # Black boxes
CELL_TYPE_TRACK = 1
CELL_TYPE_GOAL = 2
CELL_TYPE_START = 3

class RaceTrack:
    def __init__(self, track, zero_velocity_prob=0.9, max_vel=5, min_vel=0, epsilon=1):
        self.track = track
        self.wall_cells = np.argwhere(track == CELL_TYPE_WALL)
        self.goal_cells = np.argwhere(track == CELL_TYPE_GOAL)
        self.start_cells = np.argwhere(track == CELL_TYPE_START)
        self.max_vel = max_vel
        self.min_vel = min_vel
        self.colors = ['black', 'white', 'yellow', 'red']  # For plotting
        self.epsilon = epsilon
        
        self.zero_velocity_prob = zero_velocity_prob
        self.velocity_min = min_vel
        self.velocity_max = max_vel
        self.velocity_decrease_limit = -1
        self.velocity_increase_limit = 1
        
        # Q-Matrix - a 6 dimensional vector for states, velocity, and action in both directions.
        
        # Q(s, a)
        velocity_range = self.velocity_max - self.velocity_min + 1  # Minus for other direction.
        velocity_change_range = self.velocity_increase_limit  - self.velocity_decrease_limit  + 1        
        self.q = np.zeros((rt.track.shape[0], rt.track.shape[1], velocity_range, velocity_range, velocity_change_range, velocity_change_range))
        
        # Returns 
        self.Returns = defaultdict(list)

        # Initialize policy
        # NB: limit actions depending on action
        self.pi_probabilities = np.zeros((rt.track.shape[0], 
                                          rt.track.shape[1], 
                                          velocity_range, 
                                          velocity_range, 
                                          velocity_change_range, 
                                          velocity_change_range), dtype=float)

        # Initialize with equal probabilties for all possible actions        
        for y_coord in range(self.pi_probabilities.shape[0]):
            for x_coord in range(self.pi_probabilities.shape[1]):
                for y_vel in range(self.velocity_min, self.velocity_max + 1):
                    for x_vel in range(self.velocity_min, self.velocity_max + 1):
                        possible_actions = self.possible_actions((y_vel, x_vel))
                        for y_vel_change, x_vel_change in possible_actions:
                            self.pi_probabilities[y_coord, x_coord, y_vel, x_vel, y_vel_change, x_vel_change] = 1/len(possible_actions)
                                
    def policy_iteration(self):
        """
        """
        
        policy_improvement = False
        
        k=0
        while not policy_improvement:
            print('Iteration {}'.format(k))
            # Generate an episode
            G = self.generate_episode()
            
            # Append G to Returns(s, a)
            for s_a in G.keys():
                self.Returns[s_a].append(G[s_a])
            
            # Old Q(s, a)
            old_q = self.q.copy()
            
            # Calculate averages in Q(s, a)
            for s_a in self.Returns.keys():
#                print(s_a)
                self.q[eval(s_a)[0],
                       eval(s_a)[1],
                       eval(s_a)[2],
                       eval(s_a)[3],
                       eval(s_a)[4],
                       eval(s_a)[5]] = np.average(self.Returns[s_a])
            
            # Q-diff
            q_diff = abs(old_q - self.q)
            print('Q-diff: {}'.format(np.max(q_diff)))
            
            # Old policy
            old_policy = self.pi_probabilities.copy()
            
            # Update pi(a | s)
            self.update_policy()
            
            # Check if convergence
            if np.allclose(a, b, rtol=0.05):
                print('Policy iteration converged.')
                policy_improvement = True
                
            # Counter and update epsilon
            self.epsilon = 1/(1 + int(k/4))
            k += 1
            
    def update_policy(self):
        """
        """
        
        # Ranges
        velocity_range = self.velocity_max - self.velocity_min + 1  # Minus for other direction.
        velocity_change_range = self.velocity_increase_limit  - self.velocity_decrease_limit  + 1        

        # Initialize policy
        # NB: limit actions depending on action
        self.pi_probabilities = np.zeros((rt.track.shape[0], 
                                          rt.track.shape[1], 
                                          velocity_range, 
                                          velocity_range, 
                                          velocity_change_range, 
                                          velocity_change_range), dtype=float)

        # Initialize with equal probabilties for all possible actions        
        for y_coord in range(self.pi_probabilities.shape[0]):
            for x_coord in range(self.pi_probabilities.shape[1]):
                for y_vel in range(self.velocity_min, self.velocity_max + 1):
                    for x_vel in range(self.velocity_min, self.velocity_max + 1):
                        possible_actions = self.possible_actions((y_vel, x_vel))
                        greedy_action = self.greedy_action(state=[y_coord, x_coord, y_vel, x_vel],
                                                           possible_actions=possible_actions)                        
                        for y_vel_change, x_vel_change in possible_actions:
                            self.pi_probabilities[y_coord, 
                                                  x_coord, 
                                                  y_vel, 
                                                  x_vel, 
                                                  y_vel_change, 
                                                  x_vel_change] = self.epsilon_soft_policy(action=[y_vel_change, x_vel_change],
                                                                                           greedy_action=greedy_action, 
                                                                                           all_state_actions=possible_actions)         
            
    def generate_episode(self):
        """
        """
        
        crossed_finishing_line = False
        position = self.random_start_position()
        G = defaultdict(int)  # Dict. w/ G for first occurence for each s, a pair.
        
        step = 0
        while not crossed_finishing_line:
            print('-- Step {}'.format(step))
            step += 1
#            print('-- Position',  position)
            
            # Sample action
            action = self.sample_action_from_state(position)
            
            # Initiate s, a pair if not already in dict
            if str(position + action) not in G.keys():                
                G[str(position + action)] = 0   

            # Append -1 reward to all s, a pairs
            G = {key_: val_ - 1 for key_, val_ in G.items()}


            # Old position
            old_position = position.copy()
            
            # New position
            position[0] += position[2] + action[0]
            position[1] += position[3] + action[1]
            position[2] += action[0]
            position[3] += action[1]
            
#            print('-- New position', position)

            # Check if goal if reached (is it in the projected reactangle)
            grid_states_to_check = self.get_all_grid_cells_in_projected_retcangle(current_state=[old_position[0], old_position[1]], 
                                                                                  new_state=[position[0], position[1]])
            if self.check_if_goal_is_reached(check_grid_states=grid_states_to_check):
                print('-- Goal Reached. Terminating Episode.')
                break

            # Check if car hits boundery
            if position[0] >= self.track.shape[0] or position[1] >= self.track.shape[1]:
#                print('Hit the track boundery! (outside matrix)')
                position = self.random_start_position()
                continue
            
            new_grid_position = rt.track[position[0], position[1]]
            if new_grid_position == 0:
#                print('Hit the track boundery !')
                position = self.random_start_position()
                continue
        
        return G

    def check_if_goal_is_reached(self, check_grid_states):
        """
        """

        grid_values = []

        for y, x in check_grid_states:
            if y <= self.track.shape[0] - 1 and x <= self.track.shape[1] - 1:
                grid_values.append(rt.track[y, x])

        if 2 in grid_values:
            return True
        else:
            return False
    
    def get_all_grid_cells_in_projected_retcangle(self, current_state, new_state):
        """
        """
        y_coord_current = current_state[0]
        x_coord_current = current_state[1]

        y_diff = new_state[0] - current_state[0]
        x_diff = new_state[1] - current_state[1]

        return [[y_coord_current + y, x_coord_current + x] for y in range(0, y_diff + 1) for x in range(0, x_diff + 1)]


    def sample_action_from_state(self, state):
        """
        """
        
        # Action coordinates in probability matrix
        array = np.array(['(0, 0)', 
                          '(0, 1)', 
                          '(0, 2)', 
                          '(1, 0)', 
                          '(1, 1)', 
                          '(1, 2)',
                          '(2, 0)',
                          '(2, 1)',
                          '(2, 2)'])
        
        # Randomly pick action
#        print('---- ! {}'.format(rt.pi_probabilities[state[0], state[1], state[2], state[3]].flatten()))
        a = numpy_random.choice(array, 
                                size=1,
                                p=rt.pi_probabilities[state[0], state[1], state[2], state[3]].flatten())        
        a = eval(list(a)[0])        
        
        return self.center_axis_around_zero(coordinates=a, list_range=range(3))

    def center_axis_around_zero(self, coordinates, list_range):
        """
        """
        return [x - len(list_range) if x > len(list_range)/2 else x for x in coordinates]

    def greedy_action(self, state, possible_actions):
        """
        """
        
        q_max = 0
        greedy_action = possible_actions[0]
        
        for action in possible_actions:
            value = self.q[state[0], state[1], state[2], state[3], action[0], action[1]]
            if value > q_max:
                q_max = value
                greedy_action = action
                    
        return self.center_axis_around_zero(coordinates=greedy_action, list_range=range(3))

    def epsilon_soft_policy(self, action, greedy_action, all_state_actions):
        """
        """
        if action == greedy_action:
            return 1 - self.epsilon + self.epsilon/len(all_state_actions)
        
        return self.epsilon/len(all_state_actions)

    def random_start_position(self):
        """
        """
        
        grid_position = list(random.choice(np.argwhere(self.track==3)))
        velocity = [0, 0]
        
        return  grid_position + velocity 
    
    def possible_actions(self, velocity):
        """
        Credit: Andreas
        """
        actions = [[a_x, a_y] for a_x in range(-1, 2) for a_y in range(-1, 2)]
        legal_actions = []
        
        v_x, v_y = velocity
        
        # Discard illegal actions
        for a in actions:
            a_x, a_y = a
            # Cannot go above speed limit in any x direction
            if v_x + a_x < self.min_vel or v_x + a_x > self.max_vel:
                continue
            # Cannot go above speed limit in any y direction
            if v_y + a_y < self.min_vel or v_y + a_y > self.max_vel:
                continue
            # Cannot noop
            if v_x + a_x == 0 and v_y + a_y == 0:
                continue
            legal_actions.append(a)
            
        return legal_actions
                
    @classmethod
    def from_csv(cls, file_path):
        
        file_path = os.path.join(os.getcwd(), file_path)
        
        track = genfromtxt(file_path, delimiter=',')
        track = np.flip(track, axis=0)
        
        return cls(track) 
                    

In [824]:
rt = RaceTrack.from_csv("../racetracks/map1.csv")

In [826]:
rt.track.shape

(32, 16)

In [825]:
rt.generate_episode()

-- Step 0
-- Position [0, 7, 0, 0]
-- New position [1, 7, 1, 0]
-- Step 1
-- Position [1, 7, 1, 0]
-- New position [2, 7, 1, 0]
-- Step 2
-- Position [2, 7, 1, 0]
-- New position [3, 8, 1, 1]
-- Step 3
-- Position [3, 8, 1, 1]
-- New position [3, 9, 0, 1]
Hit the track boundery !
-- Step 4
-- Position [0, 8, 0, 0]
-- New position [0, 9, 0, 1]
Hit the track boundery !
-- Step 5
-- Position [0, 3, 0, 0]
-- New position [0, 4, 0, 1]
-- Step 6
-- Position [0, 4, 0, 1]
-- New position [1, 5, 1, 1]
-- Step 7
-- Position [1, 5, 1, 1]
-- New position [3, 5, 2, 0]
-- Step 8
-- Position [3, 5, 2, 0]
-- New position [4, 6, 1, 1]
-- Step 9
-- Position [4, 6, 1, 1]
-- New position [6, 6, 2, 0]
-- Step 10
-- Position [6, 6, 2, 0]
-- New position [9, 7, 3, 1]
-- Step 11
-- Position [9, 7, 3, 1]
-- New position [12, 9, 3, 2]
Hit the track boundery !
-- Step 12
-- Position [0, 3, 0, 0]
-- New position [1, 4, 1, 1]
-- Step 13
-- Position [1, 4, 1, 1]
-- New position [1, 5, 0, 1]
-- Step 14
-- Position [

-- Step 329
-- Position [0, 7, 0, 0]
-- New position [0, 8, 0, 1]
-- Step 330
-- Position [0, 8, 0, 1]
-- New position [0, 9, 0, 1]
Hit the track boundery !
-- Step 331
-- Position [0, 5, 0, 0]
-- New position [1, 6, 1, 1]
-- Step 332
-- Position [1, 6, 1, 1]
-- New position [2, 6, 1, 0]
-- Step 333
-- Position [2, 6, 1, 0]
-- New position [3, 7, 1, 1]
-- Step 334
-- Position [3, 7, 1, 1]
-- New position [5, 7, 2, 0]
-- Step 335
-- Position [5, 7, 2, 0]
-- New position [7, 7, 2, 0]
-- Step 336
-- Position [7, 7, 2, 0]
-- New position [10, 7, 3, 0]
-- Step 337
-- Position [10, 7, 3, 0]
-- New position [12, 7, 2, 0]
-- Step 338
-- Position [12, 7, 2, 0]
-- New position [13, 8, 1, 1]
-- Step 339
-- Position [13, 8, 1, 1]
-- New position [15, 10, 2, 2]
Hit the track boundery !
-- Step 340
-- Position [0, 8, 0, 0]
-- New position [1, 9, 1, 1]
Hit the track boundery !
-- Step 341
-- Position [0, 6, 0, 0]
-- New position [1, 6, 1, 0]
-- Step 342
-- Position [1, 6, 1, 0]
-- New position [3, 6,

-- Step 647
-- Position [0, 5, 0, 0]
-- New position [1, 5, 1, 0]
-- Step 648
-- Position [1, 5, 1, 0]
-- New position [2, 6, 1, 1]
-- Step 649
-- Position [2, 6, 1, 1]
-- New position [3, 8, 1, 2]
-- Step 650
-- Position [3, 8, 1, 2]
-- New position [4, 10, 1, 2]
Hit the track boundery !
-- Step 651
-- Position [0, 8, 0, 0]
-- New position [0, 9, 0, 1]
Hit the track boundery !
-- Step 652
-- Position [0, 5, 0, 0]
-- New position [0, 6, 0, 1]
-- Step 653
-- Position [0, 6, 0, 1]
-- New position [0, 8, 0, 2]
-- Step 654
-- Position [0, 8, 0, 2]
-- New position [1, 11, 1, 3]
Hit the track boundery !
-- Step 655
-- Position [0, 6, 0, 0]
-- New position [0, 7, 0, 1]
-- Step 656
-- Position [0, 7, 0, 1]
-- New position [1, 7, 1, 0]
-- Step 657
-- Position [1, 7, 1, 0]
-- New position [1, 8, 0, 1]
-- Step 658
-- Position [1, 8, 0, 1]
-- New position [2, 10, 1, 2]
Hit the track boundery !
-- Step 659
-- Position [0, 3, 0, 0]
-- New position [0, 4, 0, 1]
-- Step 660
-- Position [0, 4, 0, 1]
--

-- Step 966
-- Position [0, 3, 0, 0]
-- New position [1, 4, 1, 1]
-- Step 967
-- Position [1, 4, 1, 1]
-- New position [2, 5, 1, 1]
-- Step 968
-- Position [2, 5, 1, 1]
-- New position [3, 6, 1, 1]
-- Step 969
-- Position [3, 6, 1, 1]
-- New position [4, 6, 1, 0]
-- Step 970
-- Position [4, 6, 1, 0]
-- New position [6, 6, 2, 0]
-- Step 971
-- Position [6, 6, 2, 0]
-- New position [7, 6, 1, 0]
-- Step 972
-- Position [7, 6, 1, 0]
-- New position [7, 7, 0, 1]
-- Step 973
-- Position [7, 7, 0, 1]
-- New position [7, 8, 0, 1]
-- Step 974
-- Position [7, 8, 0, 1]
-- New position [7, 9, 0, 1]
Hit the track boundery !
-- Step 975
-- Position [0, 4, 0, 0]
-- New position [1, 5, 1, 1]
-- Step 976
-- Position [1, 5, 1, 1]
-- New position [3, 6, 2, 1]
-- Step 977
-- Position [3, 6, 2, 1]
-- New position [5, 6, 2, 0]
-- Step 978
-- Position [5, 6, 2, 0]
-- New position [7, 6, 2, 0]
-- Step 979
-- Position [7, 6, 2, 0]
-- New position [10, 7, 3, 1]
-- Step 980
-- Position [10, 7, 3, 1]
-- New posit

-- Step 1267
-- Position [0, 3, 0, 0]
-- New position [1, 3, 1, 0]
-- Step 1268
-- Position [1, 3, 1, 0]
-- New position [2, 3, 1, 0]
-- Step 1269
-- Position [2, 3, 1, 0]
-- New position [4, 3, 2, 0]
-- Step 1270
-- Position [4, 3, 2, 0]
-- New position [7, 3, 3, 0]
-- Step 1271
-- Position [7, 3, 3, 0]
-- New position [11, 4, 4, 1]
-- Step 1272
-- Position [11, 4, 4, 1]
-- New position [14, 6, 3, 2]
-- Step 1273
-- Position [14, 6, 3, 2]
-- New position [18, 9, 4, 3]
Hit the track boundery !
-- Step 1274
-- Position [0, 5, 0, 0]
-- New position [1, 5, 1, 0]
-- Step 1275
-- Position [1, 5, 1, 0]
-- New position [2, 5, 1, 0]
-- Step 1276
-- Position [2, 5, 1, 0]
-- New position [4, 5, 2, 0]
-- Step 1277
-- Position [4, 5, 2, 0]
-- New position [5, 5, 1, 0]
-- Step 1278
-- Position [5, 5, 1, 0]
-- New position [7, 5, 2, 0]
-- Step 1279
-- Position [7, 5, 2, 0]
-- New position [9, 5, 2, 0]
-- Step 1280
-- Position [9, 5, 2, 0]
-- New position [12, 6, 3, 1]
-- Step 1281
-- Position [12, 6

...

-- New position [1, 7, 0, 1]
-- Step 2382
-- Position [1, 7, 0, 1]
-- New position [1, 8, 0, 1]
-- Step 2383
-- Position [1, 8, 0, 1]
-- New position [1, 10, 0, 2]
Hit the track boundery !
-- Step 2384
-- Position [0, 3, 0, 0]
-- New position [1, 4, 1, 1]
-- Step 2385
-- Position [1, 4, 1, 1]
-- New position [3, 6, 2, 2]
-- Step 2386
-- Position [3, 6, 2, 2]
-- New position [4, 9, 1, 3]
Hit the track boundery !
-- Step 2387
-- Position [0, 3, 0, 0]
-- New position [1, 3, 1, 0]
-- Step 2388
-- Position [1, 3, 1, 0]
-- New position [2, 4, 1, 1]
-- Step 2389
-- Position [2, 4, 1, 1]
-- New position [3, 4, 1, 0]
-- Step 2390
-- Position [3, 4, 1, 0]
-- New position [4, 5, 1, 1]
-- Step 2391
-- Position [4, 5, 1, 1]
-- New position [5, 7, 1, 2]
-- Step 2392
-- Position [5, 7, 1, 2]
-- New position [7, 8, 2, 1]
-- Step 2393
-- Position [7, 8, 2, 1]
-- New position [8, 9, 1, 1]
Hit the track boundery !
-- Step 2394
-- Position [0, 3, 0, 0]
-- New position [1, 3, 1, 0]
-- Step 2395
-- Position

-- Step 2647
-- Position [0, 8, 0, 0]
-- New position [1, 9, 1, 1]
Hit the track boundery !
-- Step 2648
-- Position [0, 5, 0, 0]
-- New position [0, 6, 0, 1]
-- Step 2649
-- Position [0, 6, 0, 1]
-- New position [0, 7, 0, 1]
-- Step 2650
-- Position [0, 7, 0, 1]
-- New position [0, 9, 0, 2]
Hit the track boundery !
-- Step 2651
-- Position [0, 7, 0, 0]
-- New position [1, 8, 1, 1]
-- Step 2652
-- Position [1, 8, 1, 1]
-- New position [3, 10, 2, 2]
Hit the track boundery !
-- Step 2653
-- Position [0, 6, 0, 0]
-- New position [1, 6, 1, 0]
-- Step 2654
-- Position [1, 6, 1, 0]
-- New position [2, 6, 1, 0]
-- Step 2655
-- Position [2, 6, 1, 0]
-- New position [4, 7, 2, 1]
-- Step 2656
-- Position [4, 7, 2, 1]
-- New position [7, 7, 3, 0]
-- Step 2657
-- Position [7, 7, 3, 0]
-- New position [10, 8, 3, 1]
-- Step 2658
-- Position [10, 8, 3, 1]
-- New position [12, 9, 2, 1]
Hit the track boundery !
-- Step 2659
-- Position [0, 6, 0, 0]
-- New position [1, 7, 1, 1]
-- Step 2660
-- Position 

-- New position [1, 6, 1, 2]
-- Step 2916
-- Position [1, 6, 1, 2]
-- New position [1, 8, 0, 2]
-- Step 2917
-- Position [1, 8, 0, 2]
-- New position [1, 11, 0, 3]
Hit the track boundery !
-- Step 2918
-- Position [0, 5, 0, 0]
-- New position [0, 6, 0, 1]
-- Step 2919
-- Position [0, 6, 0, 1]
-- New position [0, 8, 0, 2]
-- Step 2920
-- Position [0, 8, 0, 2]
-- New position [0, 10, 0, 2]
Hit the track boundery !
-- Step 2921
-- Position [0, 7, 0, 0]
-- New position [0, 8, 0, 1]
-- Step 2922
-- Position [0, 8, 0, 1]
-- New position [0, 9, 0, 1]
Hit the track boundery !
-- Step 2923
-- Position [0, 5, 0, 0]
-- New position [0, 6, 0, 1]
-- Step 2924
-- Position [0, 6, 0, 1]
-- New position [1, 6, 1, 0]
-- Step 2925
-- Position [1, 6, 1, 0]
-- New position [1, 7, 0, 1]
-- Step 2926
-- Position [1, 7, 0, 1]
-- New position [2, 9, 1, 2]
Hit the track boundery !
-- Step 2927
-- Position [0, 6, 0, 0]
-- New position [1, 6, 1, 0]
-- Step 2928
-- Position [1, 6, 1, 0]
-- New position [2, 6, 1, 0

-- New position [7, 7, 2, 0]
-- Step 3192
-- Position [7, 7, 2, 0]
-- New position [9, 7, 2, 0]
-- Step 3193
-- Position [9, 7, 2, 0]
-- New position [11, 8, 2, 1]
-- Step 3194
-- Position [11, 8, 2, 1]
-- New position [14, 8, 3, 0]
-- Step 3195
-- Position [14, 8, 3, 0]
-- New position [17, 8, 3, 0]
-- Step 3196
-- Position [17, 8, 3, 0]
-- New position [21, 8, 4, 0]
-- Step 3197
-- Position [21, 8, 4, 0]
-- New position [25, 8, 4, 0]
-- Step 3198
-- Position [25, 8, 4, 0]
-- New position [28, 8, 3, 0]
-- Step 3199
-- Position [28, 8, 3, 0]
-- New position [32, 8, 4, 0]
Hit the track boundery! (outside matrix)
-- Step 3200
-- Position [0, 7, 0, 0]
-- New position [1, 7, 1, 0]
-- Step 3201
-- Position [1, 7, 1, 0]
-- New position [2, 8, 1, 1]
-- Step 3202
-- Position [2, 8, 1, 1]
-- New position [2, 10, 0, 2]
Hit the track boundery !
-- Step 3203
-- Position [0, 3, 0, 0]
-- New position [1, 3, 1, 0]
-- Step 3204
-- Position [1, 3, 1, 0]
-- New position [2, 4, 1, 1]
-- Step 3205
-- Posi

{'[0, 7, 0, 0, 1, 0]': -3405,
 '[1, 7, 1, 0, 0, 0]': -3404,
 '[2, 7, 1, 0, 0, 1]': -3403,
 '[3, 8, 1, 1, -1, 0]': -3402,
 '[0, 8, 0, 0, 0, 1]': -3401,
 '[0, 3, 0, 0, 0, 1]': -3400,
 '[0, 4, 0, 1, 1, 0]': -3399,
 '[1, 5, 1, 1, 1, -1]': -3398,
 '[3, 5, 2, 0, -1, 1]': -3397,
 '[4, 6, 1, 1, 1, -1]': -3396,
 '[6, 6, 2, 0, 1, 1]': -3395,
 '[9, 7, 3, 1, 0, 1]': -3394,
 '[0, 3, 0, 0, 1, 1]': -3393,
 '[1, 4, 1, 1, -1, 0]': -3392,
 '[1, 5, 0, 1, 0, 0]': -3391,
 '[1, 6, 0, 1, 1, 1]': -3390,
 '[2, 8, 1, 2, 1, 0]': -3389,
 '[0, 6, 0, 0, 1, 1]': -3388,
 '[1, 7, 1, 1, 1, -1]': -3387,
 '[3, 7, 2, 0, 0, 0]': -3386,
 '[5, 7, 2, 0, 0, 1]': -3385,
 '[7, 8, 2, 1, 0, -1]': -3384,
 '[9, 8, 2, 0, 0, 0]': -3383,
 '[11, 8, 2, 0, 1, 0]': -3382,
 '[14, 8, 3, 0, 1, 0]': -3381,
 '[18, 8, 4, 0, -1, 0]': -3380,
 '[21, 8, 3, 0, 0, 1]': -3379,
 '[3, 8, 1, 1, 0, -1]': -3375,
 '[4, 8, 1, 0, 1, 0]': -3374,
 '[6, 8, 2, 0, 1, 0]': -3373,
 '[9, 8, 3, 0, 1, 0]': -3372,
 '[13, 8, 4, 0, 0, 1]': -3371,
 '[1, 7, 1, 1, 1, 0]': -33

In [818]:
rt.policy_iteration()

Iteration 0
-- Goal Reached. Terminating Episode.
Q-diff: 5631.0
Iteration 1
-- Goal Reached. Terminating Episode.
Q-diff: 2776.5
Iteration 2
-- Goal Reached. Terminating Episode.
Q-diff: 13742.0
Iteration 3
-- Goal Reached. Terminating Episode.
Q-diff: 6466.0
Iteration 4
-- Goal Reached. Terminating Episode.
Q-diff: 6657.0
Iteration 5
-- Goal Reached. Terminating Episode.
Q-diff: 6582.5
Iteration 6
-- Goal Reached. Terminating Episode.
Q-diff: 39686.0
Iteration 7
-- Goal Reached. Terminating Episode.
Q-diff: 19153.0
Iteration 8
-- Goal Reached. Terminating Episode.
Q-diff: 17240.0
Iteration 9
-- Goal Reached. Terminating Episode.
Q-diff: 18426.5
Iteration 10
-- Goal Reached. Terminating Episode.
Q-diff: 103386.0
Iteration 11
-- Goal Reached. Terminating Episode.
Q-diff: 100823.0
Iteration 12
-- Goal Reached. Terminating Episode.
Q-diff: 156034.0
Iteration 13
-- Goal Reached. Terminating Episode.
Q-diff: 26813.333333333336
Iteration 14
-- Goal Reached. Terminating Episode.
Q-diff: 1602

KeyboardInterrupt: 

In [761]:
dict_test['a'].append(1)

In [763]:
dict_test.keys()

dict_keys(['a'])

In [567]:
a = rt.get_all_cell_values_in_projected_retcangle([29, 12],  [31, 15])

In [569]:
rt.check_if_goal_is_reached(a)

True

In [582]:
G = defaultdict(list)

In [581]:
G[str(str([1, 1, 0, 1]), str([0, 0]))] = 1

TypeError: decoding str is not supported

In [588]:
str(str([1, 1, 0, 1] + [0, 0]))

'[1, 1, 0, 1, 0, 0]'

In [514]:
a = rt.random_start_position()

In [520]:
b =  rt.sample_action_from_state([a[0], a[1], a[2], a[3]])

In [560]:
rt.track

array([[0., 0., 0., 3., 3., 3., 3., 3., 3., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,

In [543]:
a = get_all_cell_values_in_projected_retcangle([29, 14], [34, 16])

In [525]:
a = [29, 14]

In [527]:
b = [34, 16]

In [542]:
def get_all_cell_values_in_projected_retcangle(current_state, new_state):
    """
    """
    y_coord_current = current_state[0]
    x_coord_current = current_state[1]
    
    y_diff = new_state[0] - current_state[0]
    x_diff = new_state[1] - current_state[1]
    
    return [[y_coord_current + y, x_coord_current + x] for y in range(0, y_diff + 1) for x in range(0, x_diff + 1)]


In [553]:
def check_if_goal_is_reached(check_grid_states):
    """
    """
    
    grid_values = []
    
    for y, x in check_grid_states:
        if y <= rt.track.shape[0] - 1 and x <= rt.track.shape[1] - 1:
            grid_values.append(rt.track[y, x])
    
    if 2 in grid_values:
        return True
    else:
        return False

In [499]:
rt.track.shape

(32, 16)

In [ ]:
# If I'm in e.g. 1, 14, -2, 2
# And takes action -1, 0

# New position -2, 16, -3, 2


In [540]:
rt.track

array([[0., 0., 0., 3., 3., 3., 3., 3., 3., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,

In [147]:
a[0, 1] = random.choice(rt.all_possible_actions(5, 4))

In [ ]:
# Apply a Monte Carlo control method to this task to compute 
# the optimal policy from each starting state.

In [22]:
# Track bounderies:

race_track = {'[0, 2]': '[0, 6]',
              '[3, 9]': '[-1, 6]',
              '[10, 17]': '[-2, 6]',
              '[18, 24]':  '[-3, 6]',
              '[25, 25]': '[-3, 7]',
              '[26, 27]': '[-3, 14]',
              '[28, 28]': '[-2,  14]',
              '[29, 30]': '[-1, 14]',
              '[31, 31]': '[0, 14]'}


In [23]:
# States:
states = []
for key in race_track.keys():
    for x_coord in range(eval(key)[0], eval(key)[1] + 1):
        for y_coord in range(eval(race_track[key])[0], eval(race_track[key])[1] + 1):
            states.append([x_coord, y_coord])
            
# Start states:
start_states = [[0, 0],
                [0, 1],
                [0, 2],
                [0, 3],
                [0, 4],
                [0, 5],
                [0, 6]]

# End states:
end_states = [[x, 14] for x in range(26, 32)]

In [51]:
class RaceTrackingMDP:
    def __init__(self,
                 states,
                 start_states,
                 end_states,
                 zero_velocity_prob=0.9,
                 velocity_max=4,
                 speed_decrease_limit=-1,
                 speed_increase_limit=1):
        self.states = states
        self.start_states = start_states
        self.end_states = end_states
        self.zero_velocity_prob = zero_velocity_prob
        self.velocity_max = velocity_max
        self.speed_decrease_limit = speed_decrease_limit
        self.speed_increase_limit = speed_increase_limit
        
        self.q_values = [0 for x in self.states for a in range(0, 9 + 1)]
        self.returns = [[] for x in self.states for a in range(0, 9 + 1)]
        self.policies = [[1/9] * 6 for x in self.states for a in range(0, 9 + 1)]
        
    def actions(self, x_coord, y_coord):
        """Iterator over all actions"""
        
        for a1 in range(self.speed_decrease_limit, self.speed_increase_limit + 1):
            for a2 in range(self.speed_decrease_limit, self.speed_increase_limit + 1):
                yield a1 + x_coord, a2 + y_coord
                
    def 
                
    def policy_iteration(self):
        """Iterates over policies"""
        
        for iteration in range(0, 10):
            

In [52]:
mdp = RaceTrackingMDP(states=states, start_states=start_states, end_states=end_states)

In [16]:
a = np.zeros((2, 4, 5))


In [17]:
a

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])